# Membuat Tabel dan Upload Data #

In [ ]:
import sqlite3

conn = sqlite3.connect('eagleairlines.db')
c = conn.cursor()

# Mengaktifkan dukungan foreign key
c.execute('PRAGMA foreign_keys = ON')

c.execute('''CREATE TABLE tiket (
             nomor_tiket TEXT PRIMARY KEY,
             id_pesawat TEXT,
             model_pesawat TEXT,
             jarak_tempuh_pesawat INTEGER,
             kode_bandara TEXT,
             nama_bandara TEXT,
             kota_bandara TEXT,
             koordinat_bandara NUMBER,
             zona_waktu_bandara TEXT,
             id_penerbangan INTEGER,
             nomor_penerbangan TEXT,
             jadwal_keberangkatan DATE,
             jadwal_kedatangan DATE,
             bandara_keberangkatan TEXT,
             bandara_kedatangan TEXT,
             status_penerbangan  TEXT,
             nomor_kursi TEXT,
             kelas_kursi TEXT,
             harga_tiket NUMBER,
             kode_pemesanan TEXT,
             total_harga NUMBER,
             FOREIGN KEY (kode_pemesanan) REFERENCES pemesanan (kode_pemesanan)
             ON DELETE CASCADE ON UPDATE NO ACTION)''')

c.execute('''CREATE TABLE penumpang (
             id_penumpang INTEGER PRIMARY KEY,
             nama_penumpang TEXT)''')

c.execute('''CREATE TABLE pemesanan (
             kode_pemesanan STRING PRIMARY KEY,
             id_penumpang INTEGER,
             nomor_tiket INTEGER,
             tanggal_pemesanan DATE,
             bandara_keberangkatan STRING,
             bandara_kedatangan STRING,
             jadwal_keberangkatan DATE REFERENCES tiket (jadwal_keberangkatan),
             jadwal_kedatangan DATE REFERENCES tiket (jadwal_kedatangan),
             FOREIGN KEY (id_penumpang) REFERENCES penumpang (id_penumpang)
             ON DELETE CASCADE ON UPDATE NO ACTION,
             FOREIGN KEY (nomor_tiket) REFERENCES tiket (nomor_tiket))''')

conn.commit()
conn.close()

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import csv

MyDB = sqlite3.connect('eagleairlines.db')
cursor = MyDB.cursor()

with open('tiket.csv', 'r', encoding='latin-1') as berkas_csv:
    data_csv = csv.reader(berkas_csv, delimiter=',')
    next(data_csv)
    data = list(data_csv)
    for row in data:
        cursor.execute("INSERT INTO tiket (nomor_tiket, id_pesawat, model_pesawat, jarak_tempuh_pesawat, kode_bandara, nama_bandara, kota_bandara, koordinat_bandara, zona_waktu_bandara, id_penerbangan, nomor_penerbangan, jadwal_keberangkatan, jadwal_kedatangan, bandara_keberangkatan, bandara_kedatangan, status_penerbangan, nomor_kursi, kelas_kursi, harga_tiket, kode_pemesanan, total_harga) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", tuple(row))

MyDB.commit()
MyDB.close()

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving penumpang.csv to penumpang.csv


In [ ]:
import csv

MyDB = sqlite3.connect('eagleairlines.db')
cursor = MyDB.cursor()

with open('penumpang.csv', 'r') as berkas_csv:
    data_csv = csv.reader(berkas_csv, delimiter=',')
    next(data_csv)
    data = list(data_csv)
    for row in data:
        cursor.execute("INSERT INTO penumpang (id_penumpang, nama_penumpang) VALUES (?, ?)", tuple(row))

MyDB.commit()
MyDB.close()

# Implementasi GUI pada Pemesanan Tiket

In [ ]:
from ipywidgets import Layout, widgets, Style
from IPython.display import display
from datetime import datetime
import sqlite3

# Membuat widget input
id_penumpang_input = widgets.IntText(
    value=0,
    description='ID Penumpang:',
    disabled=False,
    style={'description_width': 'initial'}
    )


nama_penumpang_input = widgets.Text(
    value='',
    description='Nama Penumpang:',
    disabled=False,
    style={'description_width': 'initial'}
    )

tanggal_pemesanan_input = widgets.DatePicker(
    description='Tanggal Pemesanan:',
    disabled=False,
    value=datetime.now().date(),
    style={'description_width': 'initial'}
    )

bandara_keberangkatan_input = widgets.Text(
    value='',
    description='Bandara Keberangkatan:',
    disabled=False,
    style={'description_width': 'initial'}
    )

bandara_kedatangan_input = widgets.Text(
    value='',
    description='Bandara Kedatangan:',
    disabled=False,
    style={'description_width': 'initial'}
    )

# Fungsi untuk mendapatkan tiket yang tersedia berdasarkan bandara
def get_available_tickets(bandara_keberangkatan, bandara_kedatangan):
    MyDB = sqlite3.connect('eagleairlines.db')
    cursor = MyDB.cursor()
    cursor.execute("""
        SELECT bandara_keberangkatan, bandara_kedatangan, jadwal_keberangkatan, jadwal_kedatangan, nomor_kursi, kelas_kursi, harga_tiket, nomor_tiket, kode_pemesanan
        FROM tiket
        WHERE bandara_keberangkatan = ? AND bandara_kedatangan = ?
    """, (bandara_keberangkatan, bandara_kedatangan))
    tickets = cursor.fetchall()
    MyDB.close()
    return tickets

# Fungsi untuk memperbarui dropdown tiket
def update_ticket_dropdown(change):
    bandara_keberangkatan = bandara_keberangkatan_input.value
    bandara_kedatangan = bandara_kedatangan_input.value
    tickets = get_available_tickets(bandara_keberangkatan, bandara_kedatangan)
    ticket_options = [f"{ticket[0]} - {ticket[1]} - {ticket[2]} - {ticket[3]} - {ticket[4]} - {ticket[5]} - {ticket[6]} - {ticket[7]} - {ticket[8]}" for ticket in tickets]
    ticket_dropdown.options = ticket_options

# Dropdown tiket yang akan diperbarui berdasarkan input pengguna
ticket_dropdown = widgets.Dropdown(
    options=[],
    description='Pilih Tiket:',
    disabled=False
)

# Menghubungkan perubahan input bandara dengan fungsi update
bandara_keberangkatan_input.observe(update_ticket_dropdown, names='value')
bandara_kedatangan_input.observe(update_ticket_dropdown, names='value')

# Fungsi untuk menangani pemesanan
def book_ticket(button):
    id_penumpang = id_penumpang_input.value
    nama_penumpang = nama_penumpang_input.value
    tanggal_pemesanan = tanggal_pemesanan_input.value.strftime('%Y-%m-%d')
    selected_ticket = ticket_dropdown.value
    selected_ticket_split = selected_ticket.split(' - ')
    bandara_keberangkatan = selected_ticket_split[0]
    bandara_kedatangan = selected_ticket_split[1]
    jadwal_keberangkatan = selected_ticket_split[2]
    jadwal_kedatangan = selected_ticket_split[3]
    nomor_tiket = selected_ticket_split[7]
    kode_pemesanan = selected_ticket_split[8]

    # Menampilkan detail pemesanan
    print(f"Booking details:\nID Penumpang: {id_penumpang}\nNama Penumpang: {nama_penumpang}\nTanggal: {tanggal_pemesanan}\nTiket: {selected_ticket}")

    # Menyimpan pemesanan ke database
    try:
        MyDB = sqlite3.connect('eagleairlines.db')
        cursor = MyDB.cursor()
        cursor.execute("""
            INSERT INTO pemesanan (id_penumpang, nomor_tiket, tanggal_pemesanan, bandara_keberangkatan, bandara_kedatangan, kode_pemesanan, jadwal_keberangkatan, jadwal_kedatangan)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (id_penumpang, nomor_tiket, tanggal_pemesanan, bandara_keberangkatan, bandara_kedatangan, kode_pemesanan, jadwal_keberangkatan, jadwal_kedatangan))
        MyDB.commit()
        print("Pemesanan berhasil disimpan.")
    except Exception as e:
        print("Terjadi kesalahan: Tiket sudah tidak tersedia")
    finally:
        MyDB.close()

submit_button = widgets.Button(
    description='Masukkan Data',
    disabled=False,
    button_style='success',
    tooltip='Klik untuk memasukkan data pemesanan'
)
submit_button.on_click(book_ticket)

# Menampilkan widget
display(id_penumpang_input, nama_penumpang_input, tanggal_pemesanan_input, bandara_keberangkatan_input, bandara_kedatangan_input, ticket_dropdown, submit_button)

IntText(value=0, description='ID Penumpang:', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Nama Penumpang:', style=DescriptionStyle(description_width='initial'))

DatePicker(value=datetime.date(2024, 6, 6), description='Tanggal Pemesanan:', style=DescriptionStyle(descripti…

Text(value='', description='Bandara Keberangkatan:', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Bandara Kedatangan:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Pilih Tiket:', options=(), value=None)

Button(button_style='success', description='Masukkan Data', style=ButtonStyle(), tooltip='Klik untuk memasukka…

Booking details:
ID Penumpang: 9621486706
Nama Penumpang: Aurel
Tanggal: 2024-06-05
Tiket: OVB - DME - 2024-08-10 11:50:00+03 - 2024-08-10 15:15:00+03 - 1G - Business - 83,700.00 - 0005433234746 - A10280
Pemesanan berhasil disimpan.


# Tampilan Data Tiket oleh Perusahaan

In [ ]:
import pandas as pd

def display_pemesanan():
    try:
        MyDB = sqlite3.connect('eagleairlines.db')
        cursor = MyDB.cursor()
        cursor.execute("SELECT * FROM pemesanan")
        rows = cursor.fetchall()
        MyDB.close()

        # Menampilkan data sebagai DataFrame
        df = pd.DataFrame(rows, columns=[ 'kode_pemesanan', 'id_penumpang', 'nomor_tiket', 'tanggal_pemesanan', 'bandara_keberangkatan', 'bandara_kedatangan', 'jadwal_keberangkatan', 'jadwal_kedatangan'])
        display(df)
    except Exception as e:
        print("Terjadi kesalahan:", e)

# Memanggil fungsi untuk menampilkan tabel pemesanan
display_pemesanan()

,kode_pemesanan,id_penumpang,nomor_tiket,tanggal_pemesanan,bandara_keberangkatan,bandara_kedatangan,jadwal_keberangkatan,jadwal_kedatangan
0,A10280,9621486706,5433234746,2024-06-05,OVB,DME,2024-08-10 11:50:00+03,2024-08-10 15:15:00+03
1,0AA07A,7259688279,5435612620,2024-06-05,SVO,SVX,2024-07-24 12:50:00+03,2024-07-24 14:35:00+03
2,94550000,7057728660,5435473005,2024-06-05,SVO,UUS,2024-08-25 17:15:00+03,2024-08-26 02:00:00+03
3,B25D58,6159454432,5434138639,2024-06-05,SVO,KVX,2024-08-08 14:05:00+03,2024-08-08 15:05:00+03
4,E16BA5,3507627161,5432277597,2024-06-05,DME,OVB,2024-07-18 11:05:00+03,2024-07-18 14:30:00+03


# Informasi Tiket Pelanggan

In [ ]:
import sqlite3
import pandas as pd

def view_ticket_info(nomor_tiket):
    try:
        MyDB = sqlite3.connect('eagleairlines.db')
        cursor = MyDB.cursor()
        cursor.execute("SELECT * FROM pemesanan WHERE nomor_tiket = ?", (nomor_tiket,))
        row = cursor.fetchone()
        MyDB.close()

        if row:
            # Menampilkan data sebagai DataFrame
            df = pd.DataFrame([row], columns=['kode_pemesanan', 'id_penumpang', 'nomor_tiket', 'tanggal_pemesanan', 'bandara_keberangkatan', 'bandara_kedatangan', 'jadwal_keberangkatan', 'jadwal_kedatangan'])
            display(df)
        else:
            print(f"Tiket dengan nomor {nomor_tiket} tidak ditemukan.")
    except Exception as e:
        print("Terjadi kesalahan:", e)

# Meminta input nomor tiket dari pengguna dan menampilkan informasi tiket
nomor_tiket = input("Masukkan nomor tiket Anda: ")
view_ticket_info(nomor_tiket)

Masukkan nomor tiket Anda: 0005433234746


,kode_pemesanan,id_penumpang,nomor_tiket,tanggal_pemesanan,bandara_keberangkatan,bandara_kedatangan,jadwal_keberangkatan,jadwal_kedatangan
0,A10280,9621486706,5433234746,2024-06-05,OVB,DME,2024-08-10 11:50:00+03,2024-08-10 15:15:00+03
